In [2]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx


from tqdm import tqdm

In [13]:
def init_G(d, N):
    """
    Initialize the graph G with N nodes and degree d.
    """
    G = nx.random_regular_graph(d, N)
    return G

In [67]:
def initialize_survey():
    """
    Initialize a survey message.
    """
    N_possible_warnings = 16
    survey_values = np.zeros((16,1))
    survey_values[15] = 0.1
    idx = np.random.choice(1,15)
    survey_values[idx] = 0.9
    survey_values = survey_values/np.sum(survey_values)
    warning_keys = ['----','---+', '--+-', '--++', '-+--', '-+-+', '-++-', '-+++', '+---', '+--+', '+-+-', '+-++', '++--', '++-+', '+++-', '++++'] # 16 possible warnings, first two are first row
    survey = dict(zip(warning_keys, survey_values))
    return survey, warning_keys, survey_values

In [72]:
def initialize_surveys(G):
    """
    Initialize a survey message for each edge in the graph G.
    """
    surveys = np.zeros((G.number_of_nodes(), G.number_of_nodes(), 16, 1))
    for i in list(G.nodes()):
        for j in G.neighbors(i):
            survey, warning_keys, survey_values = initialize_survey()
            surveys[i, j, :] = survey_values
            
    return surveys

In [65]:
def generate_all_possible_warnings():
    """
    Generate all possible warnings. In the same order as the survey values.
    """
    warnings = []
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):
                    warnings.append(np.array([[i, j], [k, l]]))
    return np.array(warnings)

def get_warning_index(warning):
    """
    Get the index of a warning in the list of all possible warnings.
    """
    warnings = generate_all_possible_warnings()
    for i in range(len(warnings)):
        if np.array_equal(warning, warnings[i]):
            return i
    return -1

# Warning Propagation code

In [57]:
def gen_configurations(d):
    """
    returns all possible configurations of d-1 neighbours
    """
    return np.array(np.meshgrid(*[[0, 1]] * (d-1))).T.reshape(-1, d-1)

def respect_rule(rule, i, j, rest_config):
        outer_density=j+np.sum(rest_config)
        if rule[outer_density]=='0':
            return True if i==0 else False
        elif rule[outer_density]=='1':
            return True if i==1 else False
        elif rule[outer_density]=='+':
            return True
        elif rule[outer_density]=='-':
            return False
        
def neighbouring_warnings_allow(sigma_i, configuration, neighbouring_warnings):
    """
    returns True if the neighbouring warnings allow the configuration, False otherwise
    """
    
    for k, warning in enumerate(neighbouring_warnings):
        sigma_k = configuration[k]
        if warning[sigma_k, sigma_i] == 0:
            return False

    return True


def warning_config_is_fixed_point(warning_index, config, rule):
    """
    Check if a warning configuration is a fixed point of a rule.
    """
    warning_list = generate_all_possible_warnings()
    warning = warning_list[warning_index]
    new_warning = np.zeros((2, 2))
    for sigma_i in range(2):
        for sigma_j in range(2):
            configurations = gen_configurations(len(config)+1)
            for configuration in configurations:
                if respect_rule(rule, sigma_i, sigma_j, configuration) and neighbouring_warnings_allow(sigma_i, configuration, config):
                    new_warning[sigma_i, sigma_j] = 1
                    break
    
    if np.array_equal(new_warning, warning):
        return True
    else:
        return False


# 3. Fow now, hard code for d = 3

In [58]:
def generate_warning_configurations():
    warning_configs = []
    warnings = generate_all_possible_warnings()
    for i in range(16):
        for j in range(16):
                warning_configs.append([warnings[i], warnings[j]])
    return warning_configs

In [59]:
def survey_propagation(G, rule, num_iters=100):
    surveys = initialize_surveys(G)
    warning_configs = generate_warning_configurations()
    for _ in tqdm(range(num_iters)):
        for i in tqdm(list(G.nodes())):
            for j in list(G.neighbors(i)):
                for warning_idx in range(16):
                    update_sum = 0
                    for config in warning_configs:
                        if warning_config_is_fixed_point(warning_index=warning_idx, config=config, rule=rule):
                            update_prod = 1
                            for k in list(G.neighbors(i)):
                                config_idx = 0
                                if k != j:
                                    update_prod *= surveys[k, i, get_warning_index(config[config_idx])]
                                    config_idx += 1
                            update_sum += update_prod
                    surveys[i, j, warning_idx] = update_sum
                surveys[i, j, :] = surveys[i, j, :]/np.linalg.norm(surveys[i, j, :])

        ## add convergence condition
    return surveys

In [45]:
G = init_G(3, 100)
rule = ['1', '0', '0', '0']
surveys = survey_propagation(G, rule, num_iters=100)
print(surveys[0, 1, :])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [69]:
def fixed_point_update(config, rule):
    """
    Check if a warning configuration is a fixed point of a rule.
    """
    new_warning = np.zeros((2, 2))
    for sigma_i in range(2):
        for sigma_j in range(2):
            configurations = gen_configurations(len(config)+1)
            for configuration in configurations:
                if respect_rule(rule, sigma_i, sigma_j, configuration) and neighbouring_warnings_allow(sigma_i, configuration, config):
                    new_warning[sigma_i, sigma_j] = 1
                    break
    
    return new_warning

def survey_propagation_eff(G, rule, num_iters=100):
    surveys = initialize_surveys(G)
    warning_configs = generate_warning_configurations()
    warning_list = generate_all_possible_warnings()
    for _ in tqdm(range(num_iters)):
        for i in list(G.nodes()):
            for j in list(G.neighbors(i)):
                update_sum = np.zeros((16,1))
                for config in warning_configs:
                    new_warning = fixed_point_update(config, rule)
                    for warning_idx in range(16):
                        if np.array_equal(new_warning, warning_list[warning_idx]):
                            update_prod = 1
                            config_idx = 0
                            for k in list(G.neighbors(i)):
                                if k != j:
                                    update_prod *= surveys[k, i, get_warning_index(config[config_idx])]
                                    config_idx += 1
                            update_sum[warning_idx] += update_prod
                surveys[i, j, :] += update_sum
                surveys[i, j, :] = surveys[i, j, :]/np.linalg.norm(surveys[i, j, :])

        ## add convergence condition
    return surveys

In [73]:
G = init_G(3, 100)
rule = ['1', '0', '0', '0']
surveys = survey_propagation_eff(G, rule, num_iters=100)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [16:40<00:00, 10.01s/it]


In [74]:
for i in list(G.nodes()):
    for j in list(G.neighbors(i)):
        print(surveys[i, j, :])

[[1.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [1.60286373e-32]
 [7.51656054e-32]]
[[1.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [1.66367929e-32]
 [7.48673233e-32]]
[[1.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [1.66379398e-32]
 [7.48667547e-32]]
[[1.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.000

In [17]:
# import copy as cp

# obj = generate_all_possible_warnings()

# def combinations(d):
#     final = []
    
#     combination(d, final)
#     final = np.array(final)
#     return final
    
# def combination(d, array):
#     if d == 0:
#         return
#     if not array:
#         for i in range(16):
#             array.append([obj[i]])
#             combination(d-1, array[i])
#     else:
#         root = cp.deepcopy(array)
#         array.clear()
#         for i in range(16):
#             array.append(root + [obj[i]])
#             combination(d-1,array[i])

# print(combinations(3).shape)

(16, 16, 16, 3, 2, 2)


In [ ]:
# def generate_warning_configurations(d, current_indices=[]): 
#     shape = [16] * d + [2, 2]
#     warning_configurations = np.zeros(shape)
#     if len(current_indices) == d:
#         return warning_configurations
#     else:
#         for i in range(16):
#             warning_configurations[current_indices + [i]] = generate_all_possible_warnings()
#         generate_warning_configurations(d, current_indices + [i])



In [23]:
shape = [16] * 3 + [2, 2]
warning_configurations = np.zeros(shape)
print(warning_configurations.shape)
print(shape)

(16, 16, 16, 2, 2)
[16, 16, 16, 2, 2]


In [17]:
G = init_G(3, 100)
surveys = initialize_surveys(G)
print(surveys)

[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [-0.07023905  0.39099749  0.03707192 ...  0.21332881  0.34100004
   -0.07379722]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
 